# NIQE and Ma et al. metrics study on satellite imagery

In this notebook we will investigate how the tile size of satellite imagery affects [NIQE](https://ieeexplore.ieee.org/document/6353522) and [Ma et al.](https://www.sciencedirect.com/science/article/pii/S107731421630203X) (short form *Ma*) scores.

We will generate random tile samples of varying size from two satellite sensors, WorldView-2 and GeoEye-1, and use an integration with MATLAB through the [MATLAB Engine API for Python](https://se.mathworks.com/help/matlab/matlab-engine-for-python.html) to evaluate *NIQE* and *Ma et al.* scores

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

from modules.matlab_metrics import *
from modules.image_utils import *
from modules.helpers import *
from modules.tile_generator import *

import plotly
from plotly import __version__
print(__version__)
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
init_notebook_mode(connected=True)

4.14.1


In [ ]:
# If study is already done then these should be set to False so that a new compute is not done
GENERATE_TILES = False
CALCULATE_NIQE_MA = False

meta = load_meta_pickle_csv('.', 'metadata_df', from_pickle=True)

SENSORS = ['WV02', 'GE01']
AREAS = ['La_Spezia', 'Toulon']

meta = subset_by_areas_sensor(meta, areas=AREAS, sensors=SENSORS)

# Path to location where individual satellite images are located
DATA_PATH = 'data/toulon-laspezia/' 
DATA_PATH_TILES = 'data/niqe-ma-study'

N_IMAGES = count_images(meta)
N_TILES = 500

SR_FACTOR = 4
MS_SIZES = [32, 48, 64, 72, 96, 120, 128, 144, 160, 168, 192, 224, 256]
PAN_SIZES = [size*SR_FACTOR for size in MS_SIZES]
PAN_AREAS = [size**2 for size in PAN_SIZES]
PAN_N_TILES = [int(N_TILES*PAN_SIZES[-1]/pan_size) for pan_size in PAN_SIZES]
[print(n_tiles) for n_tiles in PAN_N_TILES]

CLOUD_SEA_REMOVAL = False

BATCH_SIZE = 16
SHAVE_WIDTH = 4

4000
2666
2000
1777
1333
1066
1000
888
800
761
666
571
500


# Generate tiles with different tile sizes

In [ ]:
if GENERATE_TILES:
    pan_size_dir_paths = []
    for i in range(len(MS_SIZES)):
        meta = allocate_tiles(meta, by_partition=False, n_tiles_total=PAN_N_TILES[i])
        ms_size = MS_SIZES[i]
        pan_size = PAN_SIZES[i]
        pan_size_dir_path = str(DATA_PATH_TILES+'/'+str(pan_size))
        pan_size_dir_paths.append(pan_size_dir_path)
        generate_all_tiles(meta, pan_size_dir_path, ms_height_width=[ms_size, ms_size], 
                           sr_factor=SR_FACTOR, save_by_partition=False,
                           cloud_sea_removal=False,
                           print_tile_info=False, save_meta_to_disk=True)

Allocated 4000 tiles across the 62 images
Saved metadata dataframe as a pickle at data\niqe-ma-study\128\metadata_tile_allocation.pickle
Saved metadata dataframe as csv at data\niqe-ma-study\128\metadata_tile_allocation.csv
Generating 4000 without separating by train/val/test partition.
From image GE01_La_Spezia_2009_09_25_011651186010_0 - Generating 65 tiles
Shapes (4, 1898, 3272) (1, 7592, 13088)
Number of tiles discarded due to border pixels: 1

From image GE01_La_Spezia_2011_01_24_011651197010_0 - Generating 79 tiles
Shapes (4, 1994, 3275) (1, 7976, 13100)
Number of tiles discarded due to border pixels: 0

From image GE01_La_Spezia_2012_02_23_011651192010_0 - Generating 87 tiles
Shapes (4, 2138, 3512) (1, 8552, 14048)
Number of tiles discarded due to border pixels: 0

From image GE01_La_Spezia_2012_05_07_011651189010_0 - Generating 23 tiles
Shapes (4, 724, 3359) (1, 2900, 13436)
Number of tiles discarded due to border pixels: 3

From image GE01_La_Spezia_2012_05_16_011651187010_0 -

Number of tiles discarded due to border pixels: 0

From image WV02_Toulon_2014_04_06_011651052010_0 - Generating 62 tiles
Shapes (8, 1458, 2356) (1, 5832, 9424)
Number of tiles discarded due to border pixels: 0

From image WV02_Toulon_2015_11_16_011651049010_0 - Generating 58 tiles
Shapes (8, 1458, 2356) (1, 5832, 9424)
Number of tiles discarded due to border pixels: 0

From image WV02_Toulon_2016_03_14_011651064010_0 - Generating 64 tiles
Shapes (8, 1458, 2356) (1, 5832, 9424)
Number of tiles discarded due to border pixels: 0

From image WV02_Toulon_2016_03_22_011651057010_0 - Generating 90 tiles
Shapes (8, 1458, 2356) (1, 5832, 9424)
Number of tiles discarded due to border pixels: 0

From image WV02_Toulon_2016_03_25_011651060010_0 - Generating 78 tiles
Shapes (8, 1560, 2446) (1, 6239, 9784)
Number of tiles discarded due to border pixels: 1

From image WV02_Toulon_2016_03_30_011651053010_0 - Generating 65 tiles
Shapes (8, 1458, 2356) (1, 5832, 9424)
Number of tiles discarded due to b

Number of tiles discarded due to border pixels: 0

From image WV02_La_Spezia_2018_05_18_011650597010_0 - Generating 43 tiles
Shapes (8, 1960, 2691) (1, 7838, 10763)
Number of tiles discarded due to border pixels: 0

From image WV02_La_Spezia_2018_05_29_011650583010_0 - Generating 48 tiles
Shapes (8, 1928, 2815) (1, 7711, 11257)
Number of tiles discarded due to border pixels: 0

From image WV02_La_Spezia_2018_07_06_011650749010_0 - Generating 49 tiles
Shapes (8, 1893, 3109) (1, 7571, 12436)
Number of tiles discarded due to border pixels: 0

From image WV02_La_Spezia_2019_05_23_011650590010_0 - Generating 40 tiles
Shapes (8, 1893, 3109) (1, 7571, 12436)
Number of tiles discarded due to border pixels: 0

From image WV02_La_Spezia_2019_06_03_011650596010_0 - Generating 39 tiles
Shapes (8, 1909, 3134) (1, 7635, 12536)
Number of tiles discarded due to border pixels: 0

From image WV02_La_Spezia_2019_10_11_011650594010_0 - Generating 39 tiles
Shapes (8, 1893, 2557) (1, 7571, 10227)
Number of 

Number of tiles discarded due to border pixels: 0

From image WV02_La_Spezia_2012_01_10_011650582010_0 - Generating 32 tiles
Shapes (8, 1893, 3109) (1, 7571, 12436)
Number of tiles discarded due to border pixels: 0

From image WV02_La_Spezia_2012_05_17_011650585010_0 - Generating 48 tiles
Shapes (8, 2010, 3301) (1, 8040, 13204)
Number of tiles discarded due to border pixels: 0

From image WV02_La_Spezia_2013_10_31_011650588010_0 - Generating 31 tiles
Shapes (8, 1990, 3144) (1, 7959, 12576)
Number of tiles discarded due to border pixels: 0

From image WV02_La_Spezia_2013_12_08_011650589010_0 - Generating 38 tiles
Shapes (8, 1893, 3109) (1, 7571, 12436)
Number of tiles discarded due to border pixels: 0

From image WV02_La_Spezia_2014_11_20_011650595010_0 - Generating 42 tiles
Shapes (8, 1893, 3109) (1, 7571, 12436)
Number of tiles discarded due to border pixels: 0

From image WV02_La_Spezia_2015_01_05_011650746010_0 - Generating 35 tiles
Shapes (8, 1893, 3109) (1, 7571, 12436)
Number of 

Number of tiles discarded due to border pixels: 0

From image GE01_Toulon 2009_03_23_011651190010_0 - Generating 24 tiles
Shapes (4, 1478, 2387) (1, 5912, 9548)
Number of tiles discarded due to border pixels: 0

From image GE01_Toulon 2010_06_08_011651191010_0 - Generating 32 tiles
Shapes (4, 1503, 2429) (1, 6012, 9716)
Number of tiles discarded due to border pixels: 0

From image GE01_Toulon 2013_08_20_011651198010_0 - Generating 22 tiles
Shapes (4, 1552, 2508) (1, 6207, 10032)
Number of tiles discarded due to border pixels: 0

From image GE01_Toulon 2014_10_16_011651185010_0 - Generating 36 tiles
Shapes (4, 1613, 2608) (1, 6452, 10432)
Number of tiles discarded due to border pixels: 0

From image GE01_Toulon 2016_03_26_011651193010_0 - Generating 16 tiles
Shapes (4, 1634, 1510) (1, 6536, 6043)
Number of tiles discarded due to border pixels: 1

From image GE01_Toulon 2016_09_21_011651200010_0 - Generating 38 tiles
Shapes (4, 1460, 2359) (1, 5840, 9436)
Number of tiles discarded due to

...

Shapes (8, 1909, 3134) (1, 7635, 12536)
Number of tiles discarded due to border pixels: 0

From image WV02_La_Spezia_2019_10_11_011650594010_0 - Generating 13 tiles
Shapes (8, 1893, 2557) (1, 7571, 10227)
Number of tiles discarded due to border pixels: 0

From image WV02_Toulon_2010_11_04_011651051010_0 - Generating 13 tiles
Shapes (8, 1458, 2356) (1, 5832, 9424)
Number of tiles discarded due to border pixels: 0

From image WV02_Toulon_2011_05_04_011651058010_0 - Generating 15 tiles
Shapes (8, 1520, 2455) (1, 6079, 9820)
Number of tiles discarded due to border pixels: 0

From image WV02_Toulon_2011_12_22_011651063010_0 - Generating 17 tiles
Shapes (8, 1558, 2518) (1, 6232, 10071)
Number of tiles discarded due to border pixels: 0

From image WV02_Toulon_2013_03_16_011651062010_0 - Generating 11 tiles
Shapes (8, 1458, 2356) (1, 5832, 9424)
Number of tiles discarded due to border pixels: 0

From image WV02_Toulon_2013_04_07_011651056010_0 - Generating 16 tiles
Shapes (8, 1530, 2473) (1, 6

Number of tiles discarded due to border pixels: 0

From image WV02_La_Spezia_2014_11_20_011650595010_0 - Generating 8 tiles
Shapes (8, 1893, 3109) (1, 7571, 12436)
Number of tiles discarded due to border pixels: 0

From image WV02_La_Spezia_2015_01_05_011650746010_0 - Generating 16 tiles
Shapes (8, 1893, 3109) (1, 7571, 12436)
Number of tiles discarded due to border pixels: 0

From image WV02_La_Spezia_2015_03_19_011650598010_0 - Generating 9 tiles
Shapes (8, 1893, 3109) (1, 7571, 12436)
Number of tiles discarded due to border pixels: 0

From image WV02_La_Spezia_2015_03_19_011650600010_0 - Generating 11 tiles
Shapes (8, 1893, 3109) (1, 7571, 12436)
Number of tiles discarded due to border pixels: 0

From image WV02_La_Spezia_2015_04_24_011650748010_0 - Generating 10 tiles
Shapes (8, 1942, 3190) (1, 7768, 12759)
Number of tiles discarded due to border pixels: 0

From image WV02_La_Spezia_2015_05_21_011650592010_0 - Generating 15 tiles
Shapes (8, 1893, 3109) (1, 7571, 12436)
Number of ti

Shapes (4, 1634, 1510) (1, 6536, 6043)
Number of tiles discarded due to border pixels: 0

From image GE01_Toulon 2016_09_21_011651200010_0 - Generating 12 tiles
Shapes (4, 1460, 2359) (1, 5840, 9436)
Number of tiles discarded due to border pixels: 0

From image GE01_Toulon 2017_09_05_011651201010_0 - Generating 9 tiles
Shapes (4, 1618, 2615) (1, 6472, 10459)
Number of tiles discarded due to border pixels: 0

From image GE01_Toulon 2018_02_13_011651199010_0 - Generating 15 tiles
Shapes (4, 1597, 2581) (1, 6388, 10324)
Number of tiles discarded due to border pixels: 0

From image GE01_Toulon 2019_10_07_011651194010_0 - Generating 6 tiles
Shapes (4, 1623, 2622) (1, 6492, 10488)
Number of tiles discarded due to border pixels: 0

From image WV02_La_Spezia_2010_08_06_011650744010_0 - Generating 12 tiles
Shapes (8, 1893, 3109) (1, 7571, 12436)
Number of tiles discarded due to border pixels: 0

From image WV02_La_Spezia_2011_05_02_011650586010_0 - Generating 1 tiles
Shapes (8, 2001, 938) (1, 8

Shapes (4, 724, 3359) (1, 2900, 13436)
Number of tiles discarded due to border pixels: 0

From image GE01_La_Spezia_2012_05_16_011651187010_0 - Generating 3 tiles
Shapes (4, 698, 3114) (1, 2792, 12456)
Number of tiles discarded due to border pixels: 0

From image GE01_La_Spezia_2013_07_07_011651184010_0 - Generating 8 tiles
Shapes (4, 2108, 1464) (1, 8432, 5856)
Number of tiles discarded due to border pixels: 0

From image GE01_La_Spezia_2013_07_15_011651183010_0 - Generating 3 tiles
Shapes (4, 2251, 1342) (1, 9002, 5368)
Number of tiles discarded due to border pixels: 0

From image GE01_La_Spezia_2013_07_18_011651195010_0 - Generating 5 tiles
Shapes (4, 2105, 1427) (1, 8418, 5708)
Number of tiles discarded due to border pixels: 0

From image GE01_La_Spezia_2013_07_23_011651202010_0 - Generating 5 tiles
Shapes (4, 2133, 1946) (1, 8530, 7783)
Number of tiles discarded due to border pixels: 0

From image GE01_La_Spezia_2017_04_16_011651188010_0 - Generating 12 tiles
Shapes (4, 1928, 3166

Number of tiles discarded due to border pixels: 0

From image WV02_Toulon_2016_09_28_011651048010_0 - Generating 15 tiles
Shapes (8, 1458, 2356) (1, 5832, 9424)
Number of tiles discarded due to border pixels: 0

From image WV02_Toulon_2017_08_19_011651059010_0 - Generating 11 tiles
Shapes (8, 1505, 2430) (1, 6019, 9720)
Number of tiles discarded due to border pixels: 0

From image WV02_Toulon_2019_07_29_011650877010_0 - Generating 8 tiles
Shapes (8, 1458, 2356) (1, 5832, 9424)
Number of tiles discarded due to border pixels: 0

From image WV02_Toulon_2019_08_04_011650878010_0 - Generating 14 tiles
Shapes (8, 1458, 2356) (1, 5832, 9424)
Number of tiles discarded due to border pixels: 0

From image WV02_Toulon_2019_09_11_011650876010_0 - Generating 9 tiles
Shapes (8, 1458, 2356) (1, 5832, 9424)
Number of tiles discarded due to border pixels: 0

From image WV02_Toulon_2019_10_16_011650874010_0 - Generating 7 tiles
Shapes (8, 1458, 2356) (1, 5832, 9424)
Number of tiles discarded due to bord

# Loading the MATLAB Engine for Python

In [ ]:
if CALCULATE_NIQE_MA:
    matlab_engine = MatLabEngine('modules/matlab', ma=True, niqe=True, 
                                 output_dtype='uint16', shave_width=SHAVE_WIDTH)

Starting matlab.engine ...
matlab.engine started


# Pandas Dataframe to store results
`tile_path` is set as index.

In [ ]:
results = pd.DataFrame()
paths_all_tiles = [path.as_posix() for path in list_tiles_in_dir(DATA_PATH_TILES, ms_or_pan='pan')]
results['tile_path'] = paths_all_tiles
results.set_index('tile_path', inplace=True)
results

Found 18028 tiles of type pan in the directory provided


""
tile_path
data/niqe-ma-study/1024/GE01_La_Spezia_2009_09_25_011651186010_0/pan/00000.tif
data/niqe-ma-study/1024/GE01_La_Spezia_2009_09_25_011651186010_0/pan/00001.tif
data/niqe-ma-study/1024/GE01_La_Spezia_2009_09_25_011651186010_0/pan/00002.tif
data/niqe-ma-study/1024/GE01_La_Spezia_2009_09_25_011651186010_0/pan/00003.tif
data/niqe-ma-study/1024/GE01_La_Spezia_2009_09_25_011651186010_0/pan/00004.tif
...
data/niqe-ma-study/896/WV02_Toulon_2019_12_15_011650875010_0/pan/00009.tif
data/niqe-ma-study/896/WV02_Toulon_2019_12_15_011650875010_0/pan/00010.tif
data/niqe-ma-study/896/WV02_Toulon_2019_12_15_011650875010_0/pan/00011.tif


# Actual NIQE and Ma calculation

Only preprocessing step is shaving 4 pixels from the borders of every tile, since this also is done in the main experiments as well as in the 2018 PIRM SISR Challenge.

In [ ]:
if CALCULATE_NIQE_MA:
    for i, pan_size_dir_path in enumerate(pan_size_dir_paths):
        tile_size = PAN_SIZES[i]
        pan_geotiff_tile_paths = list_tiles_in_dir(pan_size_dir_path, ms_or_pan='pan')

        pan_tiles = []
        for pan_geotiff_tile_path in pan_geotiff_tile_paths:
            pan_tile = geotiff_to_ndarray(pan_geotiff_tile_path)
            pan_tiles.append(pan_tile)

        for j in range(len(pan_tiles)):
            niqes_ml, niqes_sk, mas = None, None, None
            batch_range = slice(BATCH_SIZE*j, BATCH_SIZE*(j+1))
            pan_tiles_batch = np.array(pan_tiles[batch_range])
            if pan_tiles_batch.shape[0] == 0:
                break

            tile_paths_batch = pan_geotiff_tile_paths[batch_range]

            n_tiles = pan_tiles_batch.shape[0]
            print('Tile size', tile_size, ', shape:', pan_tiles_batch.shape, ', batch:', j, 
                  ', batch size:', n_tiles, ', shave width:', matlab_engine.shave_width)

            tic = time.perf_counter()
            niqes_ml = matlab_engine.matlab_niqe_metric(pan_tiles_batch)
            toc = time.perf_counter()
            niqe_ml_sec_per_tile = (toc-tic)/n_tiles
            print('NIQE (matlab) mean:', np.mean(niqes_ml), ', sd:',  np.std(niqes_ml), 
                  ', seconds per tile:', niqe_ml_sec_per_tile)

            if tile_size > 192:
                tic = time.perf_counter()
                niqes_sk = matlab_engine.skvideo_niqe_metric(pan_tiles_batch)
                toc = time.perf_counter()
                niqe_sk_sec_per_tile = (toc-tic)/n_tiles
                print('NIQE (skvideo) mean:', np.mean(niqes_sk), ', sd:',  np.std(niqes_sk), 
                      ', seconds per tile:', niqe_sk_sec_per_tile)

            tic = time.perf_counter()
            mas = matlab_engine.matlab_ma_metric(pan_tiles_batch)
            toc = time.perf_counter()
            ma_sec_per_tile = (toc-tic)/n_tiles
            print('Ma (matlab) mean:', np.mean(mas), ', sd:',  np.std(mas), 
                  ', seconds per tile:', ma_sec_per_tile)
            tile_paths_batch = [path.as_posix() for path in tile_paths_batch]
            for i, tile_path in enumerate(tile_paths_batch):
                try:
                    results.loc[tile_path, 'niqe_ml'] = niqes_ml[i]
                    results.loc[tile_path, 'niqe_ml_s_per_tile'] = niqe_ml_sec_per_tile
                except TypeError:
                    pass
                try:
                    results.loc[tile_path, 'niqe_sk'] = niqes_sk[i]
                    results.loc[tile_path, 'niqe_sk_s_per_tile'] = niqe_sk_sec_per_tile
                except TypeError:
                    pass
                try:
                    results.loc[tile_path, 'ma'] = mas[i]
                    results.loc[tile_path, 'ma_s_per_tile'] = ma_sec_per_tile
                except TypeError:
                    pass

        csv_path = pathlib.Path(DATA_PATH_TILES).joinpath('results-'+str(tile_size)+'.csv')
        results.to_csv(csv_path)
        print('Saved results dataframe as csv at', csv_path.as_posix())        

Found 4000 tiles of type pan in the directory provided
Tile size 128 , shape: (16, 128, 128, 1) , batch: 0 , batch size: 16 , shave width: 4
NIQE (matlab) mean: 39.80099351604936 , sd: 0.0011540726297272436 , seconds per tile: 0.04254935624999945
Ma (matlab) mean: 3.648574439362328 , sd: 0.6739826357379344 , seconds per tile: 11.055197293749998
Tile size 128 , shape: (16, 128, 128, 1) , batch: 1 , batch size: 16 , shave width: 4
NIQE (matlab) mean: 39.80080866033556 , sd: 0.00164232360098958 , seconds per tile: 0.009186362500003042
Ma (matlab) mean: 3.8830002174046223 , sd: 0.581484474631144 , seconds per tile: 0.946229074999998
Tile size 128 , shape: (16, 128, 128, 1) , batch: 2 , batch size: 16 , shave width: 4
NIQE (matlab) mean: 39.79974845969639 , sd: 0.0010586172325350413 , seconds per tile: 0.009297293750002211
Ma (matlab) mean: 3.4177877957401273 , sd: 0.6651020184824223 , seconds per tile: 0.9284362312500036
Tile size 128 , shape: (16, 128, 128, 1) , batch: 3 , batch size: 16 

Ma (matlab) mean: 3.795370560014519 , sd: 0.6907351497402462 , seconds per tile: 0.8976277812500086
Tile size 128 , shape: (16, 128, 128, 1) , batch: 29 , batch size: 16 , shave width: 4
NIQE (matlab) mean: 39.80064527602748 , sd: 0.0008572151927015829 , seconds per tile: 0.00864998125000227
Ma (matlab) mean: 3.4175629646797043 , sd: 0.7374757501767789 , seconds per tile: 0.9172490374999995
Tile size 128 , shape: (16, 128, 128, 1) , batch: 30 , batch size: 16 , shave width: 4
NIQE (matlab) mean: 39.800536946890155 , sd: 0.00202944874537915 , seconds per tile: 0.01010961249998843
Ma (matlab) mean: 3.8496020763624754 , sd: 0.7071646446125125 , seconds per tile: 0.8836479124999954
Tile size 128 , shape: (16, 128, 128, 1) , batch: 31 , batch size: 16 , shave width: 4
NIQE (matlab) mean: 39.8004628683447 , sd: 0.0016446234298465652 , seconds per tile: 0.009107275000005188
Ma (matlab) mean: 3.907419928083211 , sd: 0.6839485580894028 , seconds per tile: 0.8805424124999917
Tile size 128 , shap

Ma (matlab) mean: 3.7253113606699793 , sd: 0.7287459689838747 , seconds per tile: 0.8886187687500069
Tile size 128 , shape: (16, 128, 128, 1) , batch: 57 , batch size: 16 , shave width: 4
NIQE (matlab) mean: 39.800841623797474 , sd: 0.0010231720603096706 , seconds per tile: 0.00845856875000095
Ma (matlab) mean: 3.9341437754173327 , sd: 0.7202615265266159 , seconds per tile: 0.8910997500000093
Tile size 128 , shape: (16, 128, 128, 1) , batch: 58 , batch size: 16 , shave width: 4
NIQE (matlab) mean: 39.80043911656974 , sd: 0.0008516733293097583 , seconds per tile: 0.00881097499998873
Ma (matlab) mean: 3.6942913180647974 , sd: 0.725363918024547 , seconds per tile: 0.8913618374999999
Tile size 128 , shape: (16, 128, 128, 1) , batch: 59 , batch size: 16 , shave width: 4
NIQE (matlab) mean: 39.801048962081815 , sd: 0.0020201383354456104 , seconds per tile: 0.01016441250000355
Ma (matlab) mean: 4.438569033401373 , sd: 0.40020079689456367 , seconds per tile: 0.901940824999997
Tile size 128 , s

Ma (matlab) mean: 3.5798991846053134 , sd: 0.5279163663640213 , seconds per tile: 0.891803424999992
Tile size 128 , shape: (16, 128, 128, 1) , batch: 85 , batch size: 16 , shave width: 4
NIQE (matlab) mean: 39.80125068538005 , sd: 0.0005672880831241601 , seconds per tile: 0.008320112500001642
Ma (matlab) mean: 3.4528229783187876 , sd: 0.6905454105787922 , seconds per tile: 0.8901952562499957
Tile size 128 , shape: (16, 128, 128, 1) , batch: 86 , batch size: 16 , shave width: 4
NIQE (matlab) mean: 39.8003539307936 , sd: 0.0012051903143149272 , seconds per tile: 0.008590843749999522
Ma (matlab) mean: 3.3274152117073124 , sd: 0.709353176316936 , seconds per tile: 0.8927996374999907
Tile size 128 , shape: (16, 128, 128, 1) , batch: 87 , batch size: 16 , shave width: 4
NIQE (matlab) mean: 39.80004832607807 , sd: 0.0014064790951407852 , seconds per tile: 0.00862721875000716
Ma (matlab) mean: 3.5376885796863107 , sd: 0.7373028358691213 , seconds per tile: 0.8867299687500179
Tile size 128 , sh

Ma (matlab) mean: 4.02781296271985 , sd: 0.4563831556729133 , seconds per tile: 0.8964649312499944
Tile size 128 , shape: (16, 128, 128, 1) , batch: 113 , batch size: 16 , shave width: 4
NIQE (matlab) mean: 39.802461450582975 , sd: 0.0017322975281595585 , seconds per tile: 0.009130050000010215
Ma (matlab) mean: 4.114885776906297 , sd: 0.36361326204221933 , seconds per tile: 0.9003907062499934
Tile size 128 , shape: (16, 128, 128, 1) , batch: 114 , batch size: 16 , shave width: 4
NIQE (matlab) mean: 39.80310176900895 , sd: 0.001244679889715223 , seconds per tile: 0.008991793749999033
Ma (matlab) mean: 4.2654445465431845 , sd: 0.37352904681519045 , seconds per tile: 0.9020166812500179
Tile size 128 , shape: (16, 128, 128, 1) , batch: 115 , batch size: 16 , shave width: 4
NIQE (matlab) mean: 39.801776645213224 , sd: 0.0015721295682257952 , seconds per tile: 0.009058937500014963
Ma (matlab) mean: 4.2244471558094965 , sd: 0.35323441373765646 , seconds per tile: 0.9038157312500061
Tile size 

...

NIQE (matlab) mean: 6.641454083406671 , sd: 3.386365760798822 , seconds per tile: 0.16667155625054875
NIQE (skvideo) mean: 12.831654 , sd: 3.3336635 , seconds per tile: 0.14554538125048566
Ma (matlab) mean: 4.198253300050226 , sd: 0.7515270642363993 , seconds per tile: 31.845254987500084
Tile size 768 , shape: (16, 768, 768, 1) , batch: 12 , batch size: 16 , shave width: 4
NIQE (matlab) mean: 6.144821241515695 , sd: 2.7406492495457293 , seconds per tile: 0.16734274999998888
NIQE (skvideo) mean: 13.31661 , sd: 3.4903216 , seconds per tile: 0.14217127499978233
Ma (matlab) mean: 4.03952771158433 , sd: 0.7029022064482935 , seconds per tile: 31.83855500000027
Tile size 768 , shape: (16, 768, 768, 1) , batch: 13 , batch size: 16 , shave width: 4
NIQE (matlab) mean: 7.8671335249124885 , sd: 1.6185456966633038 , seconds per tile: 0.16823051249957643
NIQE (skvideo) mean: 18.971172 , sd: 3.3677428 , seconds per tile: 0.1465783437497521
Ma (matlab) mean: 3.7095953656532923 , sd: 0.611156970838977

NIQE (matlab) mean: 6.432402763739081 , sd: 2.4643598878584907 , seconds per tile: 0.16651103125059308
NIQE (skvideo) mean: 13.218965 , sd: 2.5681105 , seconds per tile: 0.1536162187494483
Ma (matlab) mean: 4.21162597773059 , sd: 0.7041324467652373 , seconds per tile: 32.6385844562501
Tile size 768 , shape: (16, 768, 768, 1) , batch: 34 , batch size: 16 , shave width: 4
NIQE (matlab) mean: 6.585356311426955 , sd: 1.6816619209043033 , seconds per tile: 0.16856284999994386
NIQE (skvideo) mean: 14.345669 , sd: 3.3432708 , seconds per tile: 0.15005818124882353
Ma (matlab) mean: 3.8202502293895897 , sd: 0.8091465652877897 , seconds per tile: 32.62603578749986
Tile size 768 , shape: (16, 768, 768, 1) , batch: 35 , batch size: 16 , shave width: 4
NIQE (matlab) mean: 6.228372739891917 , sd: 2.621915476026224 , seconds per tile: 0.16628676250002172
NIQE (skvideo) mean: 11.580999 , sd: 2.3465002 , seconds per tile: 0.15429445624977234
Ma (matlab) mean: 4.149582931290019 , sd: 0.5694883650392664 

NIQE (matlab) mean: 8.005450469923499 , sd: 3.7841014266087227 , seconds per tile: 0.27586458125006175
NIQE (skvideo) mean: 12.606256 , sd: 2.739899 , seconds per tile: 0.25815727500048524
Ma (matlab) mean: 3.8674033421139646 , sd: 0.8688769650908982 , seconds per tile: 44.56664332499895
Tile size 896 , shape: (16, 896, 896, 1) , batch: 14 , batch size: 16 , shave width: 4
NIQE (matlab) mean: 6.60575832784237 , sd: 2.455820119176066 , seconds per tile: 0.277287137500025
NIQE (skvideo) mean: 11.718068 , sd: 1.7898947 , seconds per tile: 0.25614526874960575
Ma (matlab) mean: 3.967806260528175 , sd: 0.6719996651626987 , seconds per tile: 44.487668687499536
Tile size 896 , shape: (16, 896, 896, 1) , batch: 15 , batch size: 16 , shave width: 4
NIQE (matlab) mean: 5.706284157872066 , sd: 1.2107573700366834 , seconds per tile: 0.27377523124960135
NIQE (skvideo) mean: 16.32457 , sd: 4.659311 , seconds per tile: 0.24122503750004398
Ma (matlab) mean: 4.312127557599112 , sd: 0.5315780919589159 , 

NIQE (matlab) mean: 6.521161678732725 , sd: 2.4127444175717323 , seconds per tile: 0.2750959636363074
NIQE (skvideo) mean: 12.347431 , sd: 1.4243944 , seconds per tile: 0.23895432727294974
Ma (matlab) mean: 3.906249474774947 , sd: 0.6514401514560024 , seconds per tile: 43.94239030000013
Saved results dataframe as csv at data/niqe-ma-study/results-896.csv
Found 500 tiles of type pan in the directory provided
Tile size 1024 , shape: (16, 1024, 1024, 1) , batch: 0 , batch size: 16 , shave width: 4
NIQE (matlab) mean: 7.793910254126085 , sd: 3.232480938581003 , seconds per tile: 0.3541852062517137
NIQE (skvideo) mean: 11.478483 , sd: 2.7422144 , seconds per tile: 0.3224341312507022
Ma (matlab) mean: 3.7523164497078447 , sd: 0.7979109659158562 , seconds per tile: 57.36777850625003
Tile size 1024 , shape: (16, 1024, 1024, 1) , batch: 1 , batch size: 16 , shave width: 4
NIQE (matlab) mean: 6.484132009177076 , sd: 3.066580464406769 , seconds per tile: 0.3545916812508949
NIQE (skvideo) mean: 11

Ma (matlab) mean: 4.388164312790804 , sd: 0.6188353216100411 , seconds per tile: 55.736854999999196
Tile size 1024 , shape: (16, 1024, 1024, 1) , batch: 21 , batch size: 16 , shave width: 4
NIQE (matlab) mean: 3.896837629516903 , sd: 0.6876329130135729 , seconds per tile: 0.3503591874996346
NIQE (skvideo) mean: 10.704508 , sd: 1.3305966 , seconds per tile: 0.2831663937504345
Ma (matlab) mean: 4.60322915341956 , sd: 0.4531349885451336 , seconds per tile: 55.69789212500109
Tile size 1024 , shape: (16, 1024, 1024, 1) , batch: 22 , batch size: 16 , shave width: 4
NIQE (matlab) mean: 5.666775832599879 , sd: 2.538948183904818 , seconds per tile: 0.3526716625001427
NIQE (skvideo) mean: 11.3199215 , sd: 1.3290409 , seconds per tile: 0.28732920625043334
Ma (matlab) mean: 4.255035658677615 , sd: 0.6928702793145838 , seconds per tile: 55.77556104375071
Tile size 1024 , shape: (16, 1024, 1024, 1) , batch: 23 , batch size: 16 , shave width: 4
NIQE (matlab) mean: 6.414141961960279 , sd: 2.5358140970

# Inspect results

In [ ]:
results = pd.read_csv(DATA_PATH_TILES+'/results-1024.csv')

def tile_size_from_path(row):
    p = row.iloc[0]
    p = pathlib.Path(row.iloc[0])
    tile_size = int(p.parents[2].stem)
    row['tile_size'] = tile_size
    row['shave_width'] = SHAVE_WIDTH
    row['tile_size_shaved'] = tile_size - 2*SHAVE_WIDTH
    return row
    
results = results.apply(tile_size_from_path, axis=1)
results

In [ ]:
fig = px.box(x=results['tile_size'], y=results['niqe_ml'], range_y=[0,20])
fig

In [ ]:
px.box(x=results['tile_size'], y=results['ma'])